In [242]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.action_chains import ActionChains
import time
from selenium.webdriver.common.by import By
import json
import pandas as pd

# Initialize a list to store the data for each individual page
all_data = {}

# Initialize the web driver
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))
# Initial search page URL
url = "https://samlingar.shm.se/sok?type=object&query=Vikingatid&rows=1000&offset=0"
driver.get(url)

# Wait for the page to load (you may need to adjust the sleep duration)
time.sleep(5)

# Find the cookie disclaimer button by its aria-label
cookie_disclaimer = driver.find_element(By.CSS_SELECTOR, '[aria-label="Godkänn alla kakor"]')
# Check if the cookie disclaimer button is displayed and then click it
if cookie_disclaimer.is_displayed():
    ActionChains(driver).move_to_element(cookie_disclaimer).click().perform()

# Find all the search result items
results = driver.find_elements(By.XPATH, "//a[contains(@class,'rKAserSVOTC2Sy4lh50nJg') and contains(@class,'miFVghdNq-ew7wuzMJWdDw')]")

In [243]:
# Dictionary to hold the title and its corresponding URL
titles_and_urls = {}

for result in results:
    # Extract the aria-label attribute and split it to get the title
    aria_label = result.get_attribute('aria-label')
    title_split = aria_label.split(' - Föremålsbenämning:')
    title = title_split[0].strip()

    # Extract the URL
    url = result.get_attribute('href')

    # Add the title and URL to the dictionary
    titles_and_urls[title] = url

# Dictionary to hold the data for all items
all_items_dict = {}

# Iterate over titles and URLs to fetch table data
for title, url in titles_and_urls.items():
    # Open the link of the result
    driver.get(url)
    time.sleep(3)  # wait for the page to load

    # Grab the first table on the page as the data source
    try:
        table = pd.read_html(driver.page_source)[0]
        row_dict = {row[0]: row[1] for row in table.itertuples(index=False)}
        all_items_dict[title] = row_dict
    except IndexError:
        print(f"No table found on page for {title}")
    except Exception as e:
        print(f"Error processing table for {title}: {e}")

In [244]:
driver.quit()


{
    "Stenyxa fr\u00e5n vikingatid": {
        "Museum": "Historiska museet",
        "F\u00f6rem\u00e5lsben\u00e4mning": "yxa med skafth\u00e5l",
        "Kategori": "Yxor",
        "Val\u00f6r": "10 bolognino (Notering)",
        "Material": "P\u00e4rontr\u00e4 (Kolven)",
        "Storlek": "Arkets vikning 4 mm (H\u00f6ger ben)  Bredd 35 mm (V\u00e4nster arm)",
        "Teknik": "Cama\u00efeudekor (\u00d6vre delen)",
        "Antal": "1",
        "Antal fragment": "1",
        "Datering": "1700 (cirka) \u2013 1799 (cirka) Tillverkningsdatum noteringar",
        "Tidsperiod": "1700-tal",
        "Tillverkningsplats": "Baouss\u00e9-Rouss\u00e9, Menton (Tillverkningsplats notering)",
        "Tillverkare": "Ablancourt, Nicolas Perrot d' (Tillverkare)",
        "Tidigare \u00e4gare": "Alice Babs",
        "Avbildad": "Abbas den store av Persien  Ok\u00e4nd fyndort, Sverige?  100-\u00e5rsminnet av Alexander I:s f\u00f6delse",
        "Motiv": "Andreas begravning  Agatas martyrium",
     

In [245]:

# You now have a dictionary with item titles as keys and table data (as dictionaries) as values
all_data_json = json.dumps(all_items_dict, indent=4)

In [246]:
all_data_json

'{\n    "Stenyxa fr\\u00e5n vikingatid": {\n        "Museum": "Historiska museet",\n        "F\\u00f6rem\\u00e5lsben\\u00e4mning": "yxa med skafth\\u00e5l",\n        "Kategori": "Yxor",\n        "Val\\u00f6r": "10 bolognino (Notering)",\n        "Material": "P\\u00e4rontr\\u00e4 (Kolven)",\n        "Storlek": "Arkets vikning 4 mm (H\\u00f6ger ben)  Bredd 35 mm (V\\u00e4nster arm)",\n        "Teknik": "Cama\\u00efeudekor (\\u00d6vre delen)",\n        "Antal": "1",\n        "Antal fragment": "1",\n        "Datering": "1700 (cirka) \\u2013 1799 (cirka) Tillverkningsdatum noteringar",\n        "Tidsperiod": "1700-tal",\n        "Tillverkningsplats": "Baouss\\u00e9-Rouss\\u00e9, Menton (Tillverkningsplats notering)",\n        "Tillverkare": "Ablancourt, Nicolas Perrot d\' (Tillverkare)",\n        "Tidigare \\u00e4gare": "Alice Babs",\n        "Avbildad": "Abbas den store av Persien  Ok\\u00e4nd fyndort, Sverige?  100-\\u00e5rsminnet av Alexander I:s f\\u00f6delse",\n        "Motiv": "Andrea

In [248]:
# Convert the JSON string to a Python dictionary
data_dict = json.loads(all_data_json)

# Convert the dictionary to a DataFrame
df = pd.DataFrame.from_dict(data_dict, orient='index')

# Now df is a DataFrame (table) representation of your JSON data
df.head()

,Museum,Föremålsbenämning,Kategori,Valör,Material,Storlek,Teknik,Antal,Antal fragment,Datering,...,Omnämns i katalog,Förvärvsmetod,Förvärvskälla,Fyndplats,Arkeologisk kontext,Kontextnamn,Undersökare,Undersökningsår,Relaterat föremål,Tillhörande texter
Stenyxa från vikingatid,Historiska museet,yxa med skafthål,Yxor,10 bolognino (Notering),Päronträ (Kolven),Arkets vikning 4 mm (Höger ben) Bredd 35 mm (...,Camaïeudekor (Övre delen),1,1,1700 (cirka) – 1799 (cirka) Tillverkningsdatum...,...,https://historiska.se/utstallningar/,Gåva,Historiska museet,"Plats: Ohio, San Juan County, Reapture Creek, ...","Boplats, Fatburstornet: 09876_TEST",Jättehögen,Arkeologiska forskningslaboratoriet Örebro lä...,1432 – 1448,TESTPOST_SIS_02,Typ Äldre inventarium Datum 1937 Titel SK...
Svärdsfäste av järn. Från vikingatiden.,Skoklosters slott,Svärd,Metall,NaN,Järn,Längd 190 mm Bredd 93 mm (underhjalt),NaN,1,1,800 – 1050 800-1050 EKR cirka,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Skäggyxa av järn från vikingatiden.,Skoklosters slott,Skäggyxa,Yxor,NaN,Järn,Längd 160 mm,Gjutning,1,NaN,800 800-1050EKR cirka,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Mynt av silver. Vikingatid.,Skoklosters slott,Mynt,Övriga reguljära mynt,NaN,Silver,Diameter 21 mm Tjocklek 1 mm,Prägling,1,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"Två spännbucklor. Ovala, enkelskaliga, av brons. Från vikingatiden.",Skoklosters slott,Oval spännbuckla P55 Spänne,NaN,NaN,Cu-legering Järn,Bredd 65 mm Längd 100 mm,NaN,2,2,800 – 1100 800-1050EKR cirka,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [249]:
df.to_csv('Viking_artifacts.csv')